# Data Gathering and Management Lab - 094290
## Lab 2 - Missing and Uncertain data 
#### Nitzan Shamir 206348187 & Omer Shubi 312236219

In [2]:
### Imports ###

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SQLContext
from scipy.stats import beta 
from sklearn import preprocessing
import os
from pyspark.ml.feature import Normalizer

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.elasticsearch:elasticsearch-hadoop:7.9.3 pyspark-shell'

!python -m pip install elasticsearch
from elasticsearch import Elasticsearch

from pyspark.sql.functions import date_format

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd # for small table fancy display only

import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.linalg import SparseVector, DenseVector,Vectors, VectorUDT
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import RegressionEvaluator

from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.pyplot as plt 
import numpy as np

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.mllib.evaluation import MulticlassMetrics


### Constants ###

# ES_HOST = 'da2020w-0001.eastus.cloudapp.azure.com'
ES_HOST = '10.0.0.5'
es = Elasticsearch([{'host': ES_HOST}], timeout=600000)

elastic_settings_mappings_basic = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    }
}

width = 1200
lower_thresh = 0
upper_thresh = 600
grouped_thresh = 0.01

### Helper Functions ###
@F.udf
def sample_beta(prev_val, uncertainPercentage, a, b, scale, loc, random_state):
  if uncertainPercentage > grouped_thresh:
    return float(beta.rvs(a=a ,b=b ,scale=scale, loc=loc, random_state=random_state))
  else:
    return float(prev_val)

@F.udf
def hour_group(hour):
  if hour < 4:
    return int(0) # '0-3'
  elif hour < 8:
    return int(1) # '4-7'
  elif hour < 12:
    return int(2) # '8-11'
  elif hour < 16:
    return int(3) #'12-15'
  elif hour < 20:
    return int(4) #'16-19'
  else:
    return int(5) #'20-23'
  

def column_to_index(c, df):
  indexer = StringIndexer(inputCol=c, outputCol= c + "Index")
  return indexer.fit(df).transform(df)

def column_to_one_hot(c, indexed_df):
  encoder = OneHotEncoderEstimator(inputCols=[c],outputCols=[c + "Vec"])
  return encoder.fit(indexed_df).transform(indexed_df)

def sparse_to_array(v):
  v = DenseVector(v)
  new_array = list([float(x) for x in v])
  return new_array

sparse_to_array_udf = F.udf(sparse_to_array, ArrayType(FloatType()))

to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

def write_to_elastic(df_to_upload, index_name: str, settings):
  
    if not es.indices.exists(index_name): # and overwrite:
        #es.indices.delete(index=index_name)
        es.indices.create(index=index_name, ignore=400, body=settings)

        df_to_upload.write.format("org.elasticsearch.spark.sql")\
            .option("es.resource", index_name)\
            .option("es.nodes.wan.only","true")\
            .option("es.port","9200")\
            .option("es.nodes",ES_HOST)\
            .option("es.nodes.client.only", "false")\
            .save()
    else:
        print("Index already exists")
        
def read_elastic(index, query="", scroll_size="10000", array_field=""):
    if not es.indices.exists(index):
        raise Exception("Index doesn't exist")

    return spark.read\
                .format("org.elasticsearch.spark.sql")\
                .option("es.nodes.wan.only","true")\
                .option("es.port","9200")\
                .option("es.nodes",ES_HOST)\
                .option("es.nodes.client.only", "false")\
                .option("pushdown", "true")\
                .option("es.query", query)\
                .option("es.scroll.size", scroll_size)\
                .option("es.scroll.keepalive", "120m")\
                .option("es.read.field.as.array.include", array_field)\
                .load(index)

  

# Create Aggregate data for prediction Task
def aggregate_data(data_frame, lower_thresh = 0, upper_thresh = 60):
  
  
  processed_df = data_frame.select("*",F.when(F.col("ellapsedTime") < lower_thresh, 1).when(F.col("ellapsedTime") > upper_thresh, 1).otherwise(0).alias('isUncertain'))

  processed_df = processed_df.withColumn('hourGroup', hour_group(F.col('currentHour')).cast("int")).withColumn('day', date_format("date", 'yyyy-MM-dd'))

  ### Select Relevant Columns ### 
  processed_df = processed_df.select("day", "areaId1", "distanceCovered", "ellapsedTime", "journeyPatternId", "vehicleId", 'busStop', "hourGroup", "congestion", "isUncertain")

  ### OneHot Encode areadId1 ### 
  indexed1 = column_to_index("areaId1",processed_df).drop("areaId1")
  encoded1 = column_to_one_hot("areaId1Index", indexed1)\
             .withColumn('areaIds', sparse_to_array_udf('areaId1IndexVec')).drop("areaId1IndexVec")

  num_area_id = len(encoded1.select("areaIds").head(1)[0][0])

  ### Groupby & Aggregate ###
  grouped_df = encoded1.groupBy("journeyPatternId","vehicleId", "day", "hourGroup")\
                       .agg(F.array(*[F.max(F.col("areaIds")[i]) for i in range(num_area_id)]).alias("areaIds"),\
                            F.sum("distanceCovered").alias("TotalDistance"), F.sum("ellapsedTime").alias("TotalTime"),\
                            F.countDistinct("busStop").alias("NumberOfBusStop"),\
                            (F.sum(F.col("congestion").cast("int"))/F.count("congestion")).alias("congestionPercentage"),\
                            (F.sum(F.col("isUncertain").cast("int")).alias("uncertainCount")),
                            F.count("*").alias("recordCount"),
                            (F.sum(F.col("isUncertain").cast("int"))/F.count("isUncertain")).alias("uncertainPercentage"))\
                        .withColumn("isWeekend", date_format("day", 'EEE').isin(["Sat", "Sun"]).cast("int"))\
                        .drop("day", "vehicleId")
  return grouped_df
 
  
# Process data for prediction task  - 1. OneHot Encode journeyPatternId & hourGroup; 2. Assemble as Vector  
def encode_and_assemble(data_frame, dummy=False):
  ### OneHot Encode journeyPatternId & hourGroup ###
  indexed2 = column_to_index("journeyPatternId",data_frame)
  encoded2 = column_to_one_hot("journeyPatternIdIndex", indexed2)\
              .withColumn('journeyPatternIds', sparse_to_array_udf('journeyPatternIdIndexVec'))\
              .drop("journeyPatternIdIndexVec","journeyPatternIdIndex")

  encoded2 = column_to_index("hourGroup",encoded2)
  encoded2 = column_to_one_hot("hourGroupIndex", encoded2)\
              .withColumn('hourGroups', sparse_to_array_udf('hourGroupIndexVec'))\
              .drop("hourGroupIndexVec", "hourGroupIndex")
  encoded3 = encoded2.withColumn("areaIds", to_vector("areaIds"))\
                     .withColumn("journeyPatternIds", to_vector("journeyPatternIds"))\
                     .withColumn("hourGroups", to_vector("hourGroups"))
  
  features_list = ["areaIds", "TotalDistance", "NumberOfBusStop", "isWeekend", "journeyPatternIds", "hourGroups", "TotalTime"]
  if dummy:
    features_list.append("isTripUncertain")
  assembler = VectorAssembler(
      inputCols=features_list,
      outputCol="features")

  input_data = assembler.transform(encoded3)
  
  input_data = input_data.select("*",F.when(F.col("congestionPercentage") > 0, 1.0).otherwise(0.0).alias('isCongestion'))
  
  return input_data


def recreate_warehouse_df(USE = "take"):
  # USE = "take" / 'sample' / 'full'
  ### Schema ###
  full_schema = StructType([StructField('_id',StructType([StructField('$oid',StringType(),True)]),True),
                       StructField('actualDelay',LongType(),True),
                       StructField('angle',DoubleType(),True),
                       StructField('anomaly',BooleanType(),True),
                       StructField('areaId',LongType(),True),
                       StructField('areaId1',LongType(),True),
                       StructField('areaId2',LongType(),True),
                       StructField('areaId3',LongType(),True),
                       StructField('atStop',BooleanType(),True),
                       StructField('busStop',LongType(),True),
                       StructField('calendar',StructType([StructField('$numberLong',StringType(),True)]),True),
                       StructField('congestion',BooleanType(),True),
                       StructField('currentHour',LongType(),True),
                       StructField('dateType',LongType(),True),
                       StructField('dateTypeEnum',StringType(),True),
                       StructField('delay',LongType(),True),
                       StructField('direction',LongType(),True),
                       StructField('distanceCovered',DoubleType(),True),
                       StructField('ellapsedTime',LongType(),True),
                       StructField('filteredActualDelay',LongType(),True),
                       StructField('gridID',StringType(),True),
                       StructField('journeyPatternId',StringType(),True),
                       StructField('justLeftStop',BooleanType(),True),
                       StructField('justStopped',BooleanType(),True),
                       StructField('latitude',DoubleType(),True),
                       StructField('lineId',StringType(),True),
                       StructField('loc',StructType([StructField('coordinates',ArrayType(DoubleType(),True),True),StructField('type',StringType(),True)]),True),
                       StructField('longitude',DoubleType(),True),
                       StructField('poiId',LongType(),True),
                       StructField('poiId2',LongType(),True),
                       StructField('probability',DoubleType(),True),
                       StructField('systemTimestamp',DoubleType(),True),
                       StructField('timestamp',StructType([StructField('$numberLong',StringType(),True)]),True),
                       StructField('vehicleId',LongType(),True),
                       StructField('vehicleSpeed',LongType(),True)])  


  ### Load and Transfrom df ###
  df = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile', schema=full_schema)
  if USE=='take':
    df = spark.createDataFrame(df.take(10000)).cache()
  elif USE =='sample':
    df = df.sample(False, 0.001, seed=0).cache()
  elif USE=='full':
    pass

  df = df.withColumn('timestamp', df['timestamp']['$numberLong'].cast('bigint'))\
         .withColumn('calendar', df['calendar']['$numberLong'].cast('bigint'))

  df = df.withColumn('date', F.from_unixtime((df['timestamp'])/1000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
         .withColumn('loc', df['loc']['coordinates'])\
         .drop("timestamp")\
         .withColumn('calendar', F.from_unixtime((df['calendar'])/1000000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
         .withColumn('systemTimestamp', F.to_timestamp(df['systemTimestamp']))


  df = df.drop("_id","calendar", "systemTimestamp", "probability", "poiId", "poiId2", "longitude", "latitude", "direction", "dateTypeEnum", "dateType", "currentHour", "angle", "filteredActualDelay", 'anomaly', 'gridID')

  df = df.withColumn("is_weekend", date_format("date", 'EEE').isin(["Sat", "Sun"]).cast("int"))

  df = df.withColumn("currentHour", F.hour("date"))

  df = df.withColumn('ellapsedTime',df['ellapsedTime']/1000)

  df = df.dropDuplicates()

  df = df.drop('lineId')
  return df


# Load from Elastic
def load_data():
#   ES_HOST = 'da2020w-0001.eastus.cloudapp.azure.com'
  ES_HOST = "10.0.0.5"

  es = Elasticsearch([{'host': ES_HOST}], timeout=600000)
  df = read_elastic('dublin-buses-main-df')
  areaId_df = read_elastic('dublin-buses-area-id')
  df = df.join(areaId_df, on='areaId')
  return df


def evaluate_model(predictions, name):
  predictionAndTarget = predictions.select("prediction", "isCongestion")
  
  metrics = MulticlassMetrics(predictionAndTarget.rdd)
   
  # Get metrics
  acc = metrics.accuracy
  Recall= metrics.recall(1)

  print(f"Accuracy {name}: {acc}")
  print(f"Recall {name}: {Recall}")

def regression_task(data, maxIter, train_ratio, USE='full'):

  # regression Model
  lr = LogisticRegression(featuresCol='features', labelCol='isCongestion').setMaxIter(maxIter)
  
  if USE == 'full':
    train, test = data.randomSplit([train_ratio, 1-train_ratio], seed=12345)
  else: # USE=='take'
    train, test = (spark.createDataFrame(data.take(1000))).randomSplit([train_ratio, 1-train_ratio], seed=12345)

  # Fit and Predict 
  lrModel = lr.fit(train)
  predictions_test = lrModel.transform(test)
  predictions_train = lrModel.transform(train)
  
  # Create both evaluators
  evaluatorMulti = MulticlassClassificationEvaluator(labelCol="isCongestion", predictionCol="prediction")
  
  evaluate_model(predictions_train, "Train")
  evaluate_model(predictions_test, "Test")

# Part 1 -  Task Definition

In this project we look at the following binary classification task:

**Predict whether a bus trip includes congestion.**

We define 'bus trip' as a trip taken by a bus on a specific trip, from start to finish. 

Specifically, each trip is defined by the following features:

 * Numerical features:
   * Number of bus stops passed through in trip 
   * Total distance covered
   * Total trip time
 * Categorial features:
   * Whether the trip was during the weekend or not
   * Area id (all areas passed through)
   * Journy pattern id
   * Hour group
 * Label:
   * isCongestion
   
   
## Business Use Case

First of all, by defining and computing bus trips (as opposed to the given bus sensor log records) we allow decision makers in general to focus on more abstract and meaningful tasks.

The task we choose can be incorporated as part of a bigger system. By providing the above features to the model, it predicts if there is a Congestion during the trip. 
* So, for example, one can use the correlations, and causal effects that the model has learned and improve the real world system better.
* Additionally, when planning a new line, route planners can provide input (assumed and computed by real world knowledge) and predict the probability of a congestion during the trip. Allowing for better decision making.  
* Note: a harder and more accurate task can be to predict the percentage of congestion during the trip, however we focus on the simpler case.

## Feature Explanations
- We define 'isCongestion' as 1 if the congestion percentage > 0 and 0 if congestion percentage = 0

- We define 'congestion percentage' as the percentage of congestion records divided by total number of records in a trip. Most records are with interval of 20 seconds so this corresponds to roughly the percentage of trip time in congestion.

- We partition the data into trips by aggregating (groupBy) over JournyPatternId, vehicleId, date and hourGroup.

- We define hourGroup as the part of day (EarlyMorning (4-7), LateMorning(8-11), EarlyNoon(12-15), Afternoon(16-19), Evening(20-23), Night(0-3)).
  Reasons behind using hourGroup to partition the trips- 

  * Partitioning only by JournyPatternId, vehicleId and date is not sufficient.
  We figured this out by looking at a specific bus, and the times at which it passed through a specific bus stop. 
  We saw that for example bus 33476 passed through bus stop 354 on July 24th 2017 at around 8am, 14pm and 20pm. 
  From this we conclude that a bus can have more than one trips on the same route on the same day. This makes sense with our understanding of buses.

  * We also conclude that partitioning by the part of day is the best way to do so, as we see, from the given data.

  * We make an assumption that for the timeframes we selected there is only one bus trip in each timeframe.
  With better understanding of the real world, e.g. by external data, this parameter can be tuned. 
    
    * In practice, some trips had only a few records - think of a trip that starts at 7:58AM and ends at 9:00AM it will be split into two trips by our partitioning method. Therefore we discard trips with less than 20 records. 
  
- We computed the number of bus stops by counting all the **unique** bus stops passed through in a trip. This is as we saw that buses report several records for each bus stop.

- We define the  Area id as binary array contains 1 in the entries corresponds to the areas passed through in the bus trip, and 0 in the all othe entries.

- Note, as the data needed for this task is different than what we prepared in task 1, we adjust the data and data warehouse accordingly.
  
   
## Relation To Insights From Task 1
 
- Elapsed time and distance are computed based on the previous record and not from start. Thus, if we sum these values from an aggregation of records we get a trip. 

- Together with the fact that journey pattern id defines a bus route, this allowed us and lead us to focus on bus trips. 

- we know the units of elapsed time and distance. 

- Additionally, as we identified that areaId relates to different areas on the map we are able to use this knowledge and incorporate the areas that a buses passes through as a bus trip feature.
  * As each areaIdX is a further partition of areaIdX-1, we allow ourselves to focus specifically on areaId1, which provides information about the trip but that is not too detailed.

## Prediction Model

The model we use is a **Logistic Regression** model, and the metrics we focus on are **Accuracy** and **Recall**.

General process to run the model and evaluate:
1. Load the aggregate data from the Elastic.

2. Encode journeyPatternId and hourGroup features into One hot vector.

3. Assemble the features into a feature vector.

4. Split the data into train and test sets. 

5. Train the model on the train set.

6. Evaluate it on both sets, seperately, by the accuracy and recall measures.

Note:
* We assume that the required assumptions for a LR model hold, as this is not the focus of the task.

* We choose to look on the model accuracy to get general knowlege on the performance of the model.

* We choose to look also on the Recall of class 1 (isCongestion=1) as it focuses on the correctness of the model on bus trips that include congestion. We think that the relevancy in practice in real world situations of our task, is better expressed by the recall measure.

* The data is inbalanced (~20 of bus trips are trips with isCongestion=1, and ~80 are bus trips with isCongestion=0), we take it into consideration in the result analysis.

## Prediction Based On Original Data

In this part we run the model on the all data, including records with uncertainty.

We compare the results between the different variations, including ways to handle the uncertain data, and provide a discussing in part 3 of this task.

In [5]:
df = recreate_warehouse_df(USE = "full")

In [6]:
grouped_df = aggregate_data(df, lower_thresh = lower_thresh, upper_thresh = upper_thresh)

write_to_elastic(df_to_upload=grouped_df, index_name="dublin-buses-grouped-df-full", settings=elastic_settings_mappings_basic)
# write_to_elastic(df_to_upload=grouped_df, index_name="dublin-buses-grouped-df-mini2", settings=elastic_settings_mappings_basic)

In [7]:
# grouped_df = read_elastic('dublin-buses-grouped-df-mini2',array_field="areaIds")
grouped_df = read_elastic('dublin-buses-grouped-df-full',array_field="areaIds")
grouped_df = grouped_df.where(grouped_df['recordCount'] > 20)

In [8]:
display(grouped_df)

NumberOfBusStop,TotalDistance,TotalTime,areaIds,congestionPercentage,hourGroup,isWeekend,journeyPatternId,recordCount,uncertainCount,uncertainPercentage
75,22.06883,3241.0,"List(0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,5,0,00751001,173,0,0.0
83,36.73508,8357.0,"List(1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,2,0,00271001,296,1,0.0033783785
86,52.135937,39807.0,"List(1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,2,0,00271001,312,1,0.0032051282
72,27.80327,4867.0,"List(1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,2,1,01451009,219,0,0.0
24,34.44526,28765.0,"List(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0)",0.0,1,0,067X1002,88,1,0.011363637
63,22.80241,6068.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,3,0,00150003,212,0,0.0
45,15.186354,4198.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,3,0,01401003,158,0,0.0
16,4.7126865,966.0,"List(0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,3,0,00181001,38,0,0.0
55,40.936966,8819.0,"List(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,2,0,00420002,243,1,0.004115226
13,4.701578,1585.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,3,0,077A1001,60,0,0.0


In [9]:
input_data = encode_and_assemble(grouped_df)

In [10]:
display(input_data.select("isCongestion", "features"))

isCongestion,features
0.0,"List(0, 604, List(1, 3, 4, 12, 13, 83, 602, 603), List(1.0, 1.0, 1.0, 22.068830490112305, 75.0, 1.0, 1.0, 3241.0))"
0.0,"List(0, 604, List(0, 1, 4, 5, 12, 13, 27, 599, 603), List(1.0, 1.0, 1.0, 1.0, 36.73508071899414, 83.0, 1.0, 1.0, 8357.0))"
0.0,"List(0, 604, List(0, 1, 4, 5, 12, 13, 27, 599, 603), List(1.0, 1.0, 1.0, 1.0, 52.13593673706055, 86.0, 1.0, 1.0, 39807.0))"
0.0,"List(0, 604, List(0, 1, 3, 6, 12, 13, 14, 23, 599, 603), List(1.0, 1.0, 1.0, 1.0, 27.80327033996582, 72.0, 1.0, 1.0, 1.0, 4867.0))"
0.0,"List(0, 604, List(2, 7, 9, 12, 13, 319, 601, 603), List(1.0, 1.0, 1.0, 34.44525909423828, 24.0, 1.0, 1.0, 28765.0))"
0.0,"List(0, 604, List(0, 1, 5, 12, 13, 22, 600, 603), List(1.0, 1.0, 1.0, 22.802410125732422, 63.0, 1.0, 1.0, 6068.0))"
0.0,"List(0, 604, List(0, 1, 12, 13, 44, 600, 603), List(1.0, 1.0, 15.18635368347168, 45.0, 1.0, 1.0, 4198.0))"
0.0,"List(0, 604, List(2, 4, 12, 13, 52, 600, 603), List(1.0, 1.0, 4.712686538696289, 16.0, 1.0, 1.0, 966.0))"
0.0,"List(0, 604, List(0, 5, 12, 13, 86, 599, 603), List(1.0, 1.0, 40.93696594238281, 55.0, 1.0, 1.0, 8819.0))"
0.0,"List(0, 604, List(0, 1, 12, 13, 31, 600, 603), List(1.0, 1.0, 4.701578140258789, 13.0, 1.0, 1.0, 1585.0))"


In [11]:
regression_task(input_data, maxIter = 10, train_ratio = 0.7, USE='full')

Accuracy Train: 0.7970904191752517
Recall Train: 0.06747116964374761
Accuracy Test: 0.7970539272562175
Recall Test: 0.06784939249414564

In [12]:
train_ratio = 0.7
myData = grouped_df.select(F.when(F.col("congestionPercentage") > 0, 1.0).otherwise(0.0).alias('isCongestion'))
train1, test1 = myData.randomSplit([train_ratio, 1-train_ratio], seed=12345)

In [13]:
train1_info = train1.groupby('isCongestion').count()
display(train1_info)

isCongestion,count
0.0,735515
1.0,190804


In [14]:
print("isCongestion = 0 :", 735515/(735515+190804) , "of train")
print("isCongestion = 1 :", 1- 735515/(735515+190804) , "of train")

isCongestion = 0 : 0.7940191230019032 of train
isCongestion = 1 : 0.20598087699809675 of train

In [15]:
test1_info = test1.groupby('isCongestion').count()
display(test1_info)

isCongestion,count
0.0,315190
1.0,81270


In [16]:
print("isCongestion = 0 :", 315190/(81270+315190) , "of test")
print("isCongestion = 1 :", 1- 315190/(81270+315190) , "of test")

isCongestion = 0 : 0.7950108459869848 of test
isCongestion = 1 : 0.2049891540130152 of test

# Part 2 - Missing\Uncertainty Detection


In this part, we aim to identify missing\uncertain data. We look at the relevant raw data features before aggregation and on bus trip features after aggregation, defining by our task from part 1.

From task 1 analysis:
  * There are no null values per se in the raw data.
  * We assume vehicleId, timestamp, JourneyPatternId, areaId1, BusStop and Congestion to be correct.
  * We know that WeekType and currentHour features don't match what we know from the timestamp info.
    * It is trivial to recompute these features based on the timestamp, so we simply do that. 
  * From EllapsedTime examination we know that there are negative values and extremely high values. This lead us to **focus on the EllapsedTime feature**.

To better understand the uncertainty, we check out the number of uncertain records, which records they are, when they occured and why.

We perform the analysis using Kibana and the data stored in our ElasticSearch Datawarehouse.

Note, similar to part 1,  as the data needed for this task is different than what we prepared in task 1, we adjust the data and data warehouse accordingly.

## Uncertainty - Which & Why? 
    
First, we look at some visualizations to understand which records are uncertain and why they occured.

To do so we look at the following visualizations:

* Figure 1- Histogram of ellapsedTime, full range
* Figure 2 - Histogram of ellapsedTime, range of -150 to 300
* Figure 3 - Sample of records with negative ellapsedTime plotted on Map
* Figure 4 - Sample of records with extremely high ellapsedTime plotted on Map

In [19]:
Image(url="https://drive.google.com/uc?id=1KQsuflxEbIEiAZSxlxIM6-9QqR3Q63Kx", width=width)

In [20]:
Image(url="https://drive.google.com/uc?id=1kvd-sdV0Lnj7C31knFCf209eNtOuyrQs",  width=width)

In [21]:
Image(url="https://drive.google.com/uc?id=14g_Bgx1UissmcIgsErg32hYo6CqnQvqY",  width=width)


In [22]:
  Image(url="https://drive.google.com/uc?id=1jz2D1GX3f5uoCGfJHR0-ocJIDQs28sc0",  width=width)

From the visualizations above we conclude the following conclusions:
* From figure 1 we can see that the ellapsedTime values of most records concentrate around 0,  and that there are a few records (but not none) with negative values or extremely high values.

* In figure 2 we took a more indepth look at ellapsedTime values and see that most of the values are betweens 0 to 40, and 20 is the most frequent.
  * We conclude that the defualt frequency for specific bus sensor report is 20 seconds. 
  
* In figure 3 we look at sequential records, including one record with **negative ellapsedTime value** (-4 seconds).
  * We can see by the timestamps (date) that record number 2 was reported after record number 1 and that record number 3 was reported after record number 2. As expected.
  * However, we can see that the calculated ellapsedTime of record number 3 was calculated respectively to record number 1, and record number 2 was caculated respectively to record number 3.
  * We conclude that those **records did not arrive in the order they were sent and that a centralized system (not the sensor) computes the relative values. Therefore, it seems to be a communication problem, e.g. the network protocol by which the sensors report the information did not maintain FIFO order.**
  
* In figure 4, we look at sequential records including one record with **extremely high ellapsedTime value** (32,735 seconds, which is roughly 9 hours).
  * We can see that record number 2 was reported directly after record number 1 and that there is no other report between them.
  * However, we can see that both records are from different days and times.
  * We conclude that on Jan 17,2018 at 23:20 the driver finished his bus trip, reported the last report for the trip (record number 1), turned off the bus sensor and came back home. The morning after, on Jan 18, 2018 at 8:30, he started his bus trip, turned on the bus sensor and reported the first report for the trip (record number 2).
  * So, the **extreme value is due to the fact that the records are computed in respect to the previous record, regardless of which trip it was part of.**

## Uncertainty - When?

Second, we look at some visualizations to understand when uncertain records occured. 

To do so we define extremly high values of ellapsedTime as ellapsedTime > 600 seconds (10 mins).

As we concluded that records are sent every 20 seconds, 10 minutes without records (for a bus trip) seems enough time to be extreme. 

The following visualizations are:
  * Figure 5 - Histogram of Count of records per vehicle id with ellapsedTime > 600.
  * Figure 6 - Histogram of record count by weeks, for records with ellapsedTime > 600.
  * Figure 7 - Histogram of Count of records per vehicle id with negative llapsedTime.
  * Figure 8 - Histogram of record count by days, for records with negative ellapsedTime.

In [25]:
Image(url="https://drive.google.com/uc?id=14l-jeM3yu7h_1WFqcavcmMj2lWSwGwLQ",width=width)

In [26]:
Image(url="https://drive.google.com/uc?id=1g5n1_omne0C2Bo8dd27E0e5kE6z_deFh",width=width)

In [27]:
Image(url="https://drive.google.com/uc?id=1MI63wKq1Yvohgr64OG5_WeVYBMOppA2Z",width=width)

In [28]:
Image(url="https://drive.google.com/uc?id=11TOuwNjkCiwRpVnWbSwr2WmrjSQmmmFE",width=width)

From the visualizations above we conclude the following conclusions:
  * From figure 5, we can see that records with **high ellapsedTime values** are distributed almost uniformly over the different vehicleIds. 
    * We conclude, that **there is no specific vehicle id that reports high ellapsedTime values**. This corresponds to our earlier findings that high values are due to over-night stoppings or long breaks. 
    * Note: we display the vehicle ids with the top 400 count values due to machine limitation.
    
  * Conclusions from Figure 5 lead us to check records with high ellapsedTime over time (weeks). From figure 6, we can see that the values are less uniform than before, but **there is no specific week with extreme reports**, and that the reports of high values of ellapsedTime occured along all weeks in the data. Note: weeks with no values in the histogram are also weeks with no records in the full data. 
  
* From figure 7, we can see that **negative ellapsedTime** values arise from all buses, pretty uniformly. Thus we conclude that **there is no specific vehicle id with much more sensor problems than the others.**

* Figure 7 conclusions lead us to check records with negative ellapsedTime over time (days). From figure 8, we can see that **there are specific days with more records with negative ellapsedTime, this might be caused by network issues.**

## Discussion & Conclusions

For our task, when looking at bus trips, the two cases we present above (1. negative ellapsedTime 2. extremely high ellapsedTime) add bias and uncertainty to the data.

In a perfect world, reports would come in FIFO order (first sent should be the first to arrive) and on the start of a trip the record details would be calculated from initial values of 0.

Therefore, we aim to fix this bias and uncertainty. To do so we first define records as uncertain records by the following:

Following our analysis, we define each record as "uncertain" if the ellapsedTime > 600 or ellapsedTime < 0.

Note: we are aware that in the negative ellapsedTime case, sequential records that follow uncertain records might contain uncertain data as well, as they are calculated respectively to the records that arrived before them. However we choose to focus on the main uncertain records, as we defined above.

Moreover, as our task is on aggregate data looking at bus trips we define bus trip as "uncertain" if the bus trip contains more than 1% uncertain records.
We choose this threshold as it corresponds to 1-2 uncertain records per trip on average. As records with uncertainty resulting from high ellapsedTime values can have a high effect on the computed trips this threshold seems reasonable to us. Note, 99.9986% of the uncertain records are due to extreme values and the others due to negative values. 

**We identified the uncertainty in the data by exploring the ellapsedTime feature, however our conclusions lead to the understanding that the uncertainty in these type of records is in all the features that are based on the timestamp. Therefore and specifically for our task, we define the uncertainty considering the ellapsedTime and distanceCovered features. **

Missing Data Classification:

- We conclude the following conclusions based on our analysis and assumptions:

  - As we describe above, we see that there are specific days with more records that arrive at the system not in order (negative ellapsedTime case) so we conclude that the uncertainty depends on observed  values (date) but not the unobserved (uncertain) values. So it seems as if the uncertainty is **MAR**.

  - As we explained, it seems as if the first records in a trip (records with extremely high ellapsedTime) are the records that are uncertain. As the values are not initialized at the beginning of each trip, they introduce uncertainty and they depend directly on their values, leading them to be **MNAR**.

## Uncertainy - How Many? 

Finally, we aim to analyze how many of our data is uncertain.

After querying the data through Kibana, we conclude that:

1. **627,252** of the records are uncertain, which is ~0.2% of the records.
2. After aggregation, **110,206** bus trips are uncertain, which is ~8% of the bus trips.

These values seem reasonable to work with. Most of the data records/bus trips are reliable (not uncertain) but still enough of the data is uncertain, so we could compare between the different methods to deal with missing/uncertain data.

Moreover, these values are in line with our findings that the uncertainty is almost uniform over the bus trips.

# Part 3 - Handle and Improve 

In this part, we suggest 3 different ways to handle the uncertain data we present in part 2.

We make a distinction between methods that deal with the uncertainty directly on the raw data (data after preprocessing and before aggregation to bus trips), and methods that are applied on the aggregated data ( data after the aggregation into bus trips).

For each data type (raw vs aggregated) we choose the methods that are most appropriate and suitable to the type of the data. 

We choose the following methods:

* On Raw Data - Complete case analysis
* On Aggregate Data - Likelihood Based Method
* On Aggregate Data - Dummy Variable Adjustment using conditional mean imputation

Below we explain in detail what each method means and run the models.

Note: some of the methods require assumptions on the missing data type (MCAR/MAR/MNAR). If this assumptions do not realy hold in the data, the methods can still be used, but can provide inaccurate results (e.g. baised estimator).

## Method 1 :
## On Raw Data - Complete Case Analysis

First, delete records that have uncertainty in them. Defined by records with ellapsedTime < 0 or ellapsedTime > 600. 

Then, create aggregated trip data based on the raw data without the uncertain records.

Finally, run the Logistic Regression model with complete data only.

In [34]:
df1 = recreate_warehouse_df(USE = "full")
df1 = df1.where(df1.ellapsedTime.between(lower_thresh,upper_thresh)) 
grouped_df1 = aggregate_data(df1, lower_thresh = lower_thresh, upper_thresh = upper_thresh)
write_to_elastic(df_to_upload=grouped_df1, index_name="dublin-buses-grouped-df-method1", settings=elastic_settings_mappings_basic)

In [35]:
grouped_df1 = read_elastic('dublin-buses-grouped-df-method1',array_field="areaIds")
grouped_df1 = grouped_df1.where(grouped_df1['recordCount'] > 20)

In [36]:
input_data = encode_and_assemble(grouped_df1)

In [37]:
# regression_task(input_data, maxIter = 1, train_ratio = 0.7, USE='take')
regression_task(input_data, maxIter = 10, train_ratio = 0.7, USE='full')

Accuracy Train: 0.8041417184534502
Recall Train: 0.16204197700883588
Accuracy Test: 0.8040435498036523
Recall Test: 0.16297656202190808

In [38]:
train_ratio = 0.7
myData2 = grouped_df1.select(F.when(F.col("congestionPercentage") > 0, 1.0).otherwise(0.0).alias('isCongestion'))
train2, test2 = myData2.randomSplit([train_ratio, 1-train_ratio], seed=12345)

In [39]:
train2_info = train2.groupby('isCongestion').count()
display(train2_info)

isCongestion,count
0.0,735059
1.0,190692


In [40]:
print("isCongestion = 0 :", 735059/(735059+190692) , "of train")
print("isCongestion = 1 :", 1- 735059/(735059+190692) , "of train")

isCongestion = 0 : 0.7940137250729408 of train
isCongestion = 1 : 0.20598627492705923 of train

In [41]:
test2_info = test2.groupby('isCongestion').count()
display(test2_info)

isCongestion,count
0.0,314976
1.0,81260


In [42]:
print("isCongestion = 0 :", 314976/(81260+314976) , "of test")
print("isCongestion = 1 :", 1- 314976/(81260+314976) , "of test")

isCongestion = 0 : 0.7949201990732796 of test
isCongestion = 1 : 0.2050798009267204 of test

## Method 2: 
##On Aggregate Data - Likelihood Based Method

In general, handling missing data methods that are based on maximum likelihood mean to estimate the underlining distribution of the data, and then to fill in the missing values by sampling from the estimated distribution.

For this method we assumed a Beta distribution (which is a very flexible and robust distribution) and aimed to find the best beta parameters that fit our data.

First, we fit the beta distribution on the aggregated data without the uncertain records, as we want to capture the real distribution and not the distorted one caused by the uncertain records.

Then, we fill in every uncertain record (bus trip) in the aggregated data by sampling from the beta distribution that we found.

Finally, we run the Logistic Regression model on all the data.

Note, we define bus trip as "uncertain" if the bus trip contains more than 1% uncertain records.

Note, for fitting the beta distribution we use records with TotalDistance<50, as there are a few outliers left and they distort the distribution.

In [44]:
grouped_df1 = read_elastic('dublin-buses-grouped-df-method1', array_field="areaIds")
grouped_df1 = grouped_df1.where(grouped_df1['recordCount'] > 20)

In [45]:
df_time = grouped_df1.select('TotalTime').toPandas()
fig, ax = plt.subplots(2, 1, figsize=(20,16))

######### TotalTime  #########
# Find the best beta distribution
a_time, b_time, loc_time, scale_time = beta.fit(pd.DataFrame(df_time))


x_time = np.linspace(beta.ppf(0.00, a_time, b_time, scale=scale_time, loc=loc_time),
                beta.ppf(1, a_time, b_time, scale=scale_time, loc=loc_time), 100)

ax[0].plot(x_time, beta.pdf(x_time, a_time, b_time, scale=scale_time, loc=loc_time),
       'r-', lw=5, alpha=0.6, label='beta pdf')

df_time.hist(density=True, ax=ax[0], bins=20)

######### TotalDistance  #########
df_dist = grouped_df1.where(grouped_df1.TotalDistance<50).select('TotalDistance').toPandas()

a_dist, b_dist, loc_dist, scale_dist = beta.fit(df_dist)

x_dist = np.linspace(beta.ppf(0.00, a_dist, b_dist, scale=scale_dist, loc=loc_dist),
                beta.ppf(1, a_dist, b_dist, scale=scale_dist, loc=loc_dist), 100)

ax[1].plot(x_dist, beta.pdf(x_dist, a_dist, b_dist, scale=scale_dist, loc=loc_dist),
       'r-', lw=5, alpha=0.6, label='beta pdf')

df_dist.hist(density=True, ax=ax[1], bins=20)

From the above graphs we can see that the fitted beta distributions are quite suitable for the TotalTime and the TotalDistance distributions, so it's seems a good distribution to sample from.

The Beta parameters that we use to sample the data from are:

In [47]:
print(f"TotalTime - Beta Distribution with alpha={round(a_time,3)} and beta= {round(b_time,3)}")
print(f"TotalDistance - Beta Distribution with alpha={round(a_dist,3)} and beta= {round(b_dist,3)}")

TotalTime - Beta Distribution with alpha=2.121 and beta= 5.281
TotalDistance - Beta Distribution with alpha=2.986 and beta= 5.219

In [48]:
"grouped_df2 = read_elastic('dublin-buses-grouped-df-full',array_field="areaIds")
# grouped_df2 = spark.createDataFrame(grouped_df2.where(grouped_df2['recordCount'] > 20).take(1000))
grouped_df2 = grouped_df2.where(grouped_df2['recordCount'] > 20)


grouped_df2 = grouped_df2.withColumn('aTime', F.lit(a_time))\
                          .withColumn('bTime', F.lit(b_time))\
                          .withColumn('aDist', F.lit(a_dist))\
                          .withColumn('bDist', F.lit(b_dist))\
                          .withColumn('scaleTime', F.lit(scale_time))\
                          .withColumn('scaleDist', F.lit(scale_dist))\
                          .withColumn('locTime', F.lit(loc_time))\
                          .withColumn('locDist', F.lit(loc_dist))\


grouped_df2 = grouped_df2.repartition(1).withColumn("randomState", F.monotonically_increasing_id())

grouped_df2 = grouped_df2.withColumn('filledTime', sample_beta('TotalTime','uncertainPercentage','aTime', 'bTime', 'scaleTime', 'locTime', 'randomState').cast("double")).withColumn('filledDist', sample_beta('TotalDistance','uncertainPercentage','aDist', 'bDist', 'scaleDist', 'locDist', 'randomState').cast("double"))

In [49]:
display(grouped_df2.select('uncertainPercentage', "TotalTime", 'filledTime', 'TotalDistance', 'filledDist'))

uncertainPercentage,TotalTime,filledTime,TotalDistance,filledDist
0.0,3241.0,3241.0,22.06883,22.068830490112305
0.0033783785,8357.0,8357.0,36.73508,36.73508071899414
0.0032051282,39807.0,39807.0,52.135937,52.13593673706055
0.0,4867.0,4867.0,27.80327,27.80327033996582
0.011363637,28765.0,3445.204377095063,34.44526,16.300367735464867
0.0,6068.0,6068.0,22.80241,22.802410125732422
0.0,4198.0,4198.0,15.186354,15.18635368347168
0.0,966.0,966.0,4.7126865,4.712686538696289
0.004115226,8819.0,8819.0,40.936966,40.93696594238281
0.0,1585.0,1585.0,4.701578,4.701578140258789


In [50]:
grouped_df2_fixed = grouped_df2.drop("TotalDistance").withColumnRenamed("filledDist", "TotalDistance")\
       .drop("TotalTime").withColumnRenamed("filledTime", "TotalTime")
input_data2 = encode_and_assemble(grouped_df2_fixed)

In [51]:
regression_task(input_data2, maxIter = 10, train_ratio = 0.7, USE='full')

Accuracy Train: 0.796903658458911
Recall Train: 0.06903984015690634
Accuracy Test: 0.7971649094486203
Recall Test: 0.06875790973988474

## Method 3:
## On Aggregate Data - Dummy Variable Adjustment Using Conditional Mean Imputation

DVA means to substitute (plugged in) value and in also add missing data indicator feature.

In this method, we complete 'TotalTime' and 'TotalDistance' features in uncertain bus trips with the conditional mean. 
Meaning we calculate the conditional mean TotalDistance and the mean TotalTime for every journeyPatternId then for every uncertain bus trip we fill its time and distance values based on the mean values of the corresponsing journeyPatternId.

Note: for bus trips with journeyPatternId that do not exist in the data without uncertain records, we fill the values with the unconditional mean.
I.e., the mean of the TotalTime and TotalDistance features on the whole data, not by specific journeyPatternIds.

In addition we add a feature 'isTripUncertain'. For every uncertain bus trip it is equal to 1 and for all other bus trips it is set to 0.

This method allows the model to 1. use the records that had uncertain data and 2. provide it with knowledge about which records were originally uncertain and which weren't.

In [53]:
grouped_df1 = read_elastic('dublin-buses-grouped-df-method1',array_field="areaIds")
grouped_df1 = grouped_df1.where(grouped_df1['recordCount'] > 20)
# grouped_df1 = spark.createDataFrame(grouped_df1.where(grouped_df1['recordCount'] > 20).take(100))

mean_group_df = grouped_df1.groupby("journeyPatternId").agg(F.mean('TotalTime').alias('MeanTotalTime'), F.mean('TotalDistance').alias('MeanTotalDistance'))

mean_df = grouped_df1.select(F.mean("TotalTime").alias('meanTime'), F.mean("TotalDistance").alias('meanDist')).collect()
mean_time = mean_df[0]['meanTime']
mean_dist = mean_df[0]['meanDist']

In [54]:
display(mean_group_df)

journeyPatternId,MeanTotalTime,MeanTotalDistance
00271001,5350.176794565816,23.015118526329708
025A1002,3935.4084353146854,19.90108473170449
033B0001,3073.78418972332,21.58700671638896
084A1002,2675.6392572944296,14.357423555669165
025B1003,3720.7123287671234,21.119094549793086
00330006,3851.991111111111,29.975795685450237
00371002,2067.829411764706,15.677291291601518
017A1001,3507.7750692520776,21.608268894226267
00701002,2146.9240506329115,19.49729012176866
01511001,4737.319978121154,22.057625599915607


In [55]:
print("mean_dist", mean_dist, "mean_time", mean_time)

mean_dist 19.882581645226097 mean_time 4237.813437651051

In [56]:
grouped_df3 = read_elastic('dublin-buses-grouped-df-full',array_field="areaIds")
# grouped_df3 = spark.createDataFrame(grouped_df3.where(grouped_df3['recordCount'] > 20).take(100))
grouped_df3 = grouped_df3.where(grouped_df3['recordCount'] > 20)

grouped_df3 = grouped_df3.join(mean_group_df, on='journeyPatternId', how='left')

grouped_df3 = grouped_df3.select("*",F.when(F.col("uncertainPercentage") > grouped_thresh, F.col("MeanTotalTime")).otherwise(F.col("TotalTime")).alias('filledTime'),\
                                 F.when(F.col("uncertainPercentage") > grouped_thresh, F.col("MeanTotalDistance")).otherwise(F.col("TotalDistance")).alias('filledDist'),\
                                F.when(F.col("uncertainPercentage") > grouped_thresh, 1.0).otherwise(0.0).alias('isTripUncertain'))

grouped_df3 = grouped_df3.fillna( { 'filledDist': mean_dist, 'filledTime':mean_time })

In [57]:
display(grouped_df3.select("journeyPatternId", "filledTime", "isTripUncertain", "filledDist", "uncertainPercentage"))

journeyPatternId,filledTime,isTripUncertain,filledDist,uncertainPercentage
00271001,8357.0,0,36.73508071899414,0.0033783785
00271001,39807.0,0,52.13593673706055,0.0032051282
00271001,7568.0,0,26.47331428527832,0.0034602077
00271001,7221.0,0,26.433839797973633,0.0034843206
00271001,6523.0,0,31.163270950317383,0.0
00271001,6884.0,0,27.393417358398438,0.0053475937
00271001,7473.0,0,29.608427047729492,0.0
00271001,5300.0,0,20.455768585205078,0.0
00271001,7926.0,0,27.96272850036621,0.0032467532
00271001,5806.0,0,24.234407424926758,0.0


In [58]:
grouped_df3_fixed = grouped_df3.drop("TotalDistance").withColumnRenamed("filledDist", "TotalDistance")\
       .drop("TotalTime").withColumnRenamed("filledTime", "TotalTime")
input_data3 = encode_and_assemble(grouped_df3_fixed, dummy=True)

In [59]:
regression_task(input_data3, maxIter = 10, train_ratio = 0.7, USE='full')

Accuracy Train: 0.7977002080897876
Recall Train: 0.07407952484823001
Accuracy Test: 0.7968390073939433
Recall Test: 0.07283149390728315

## Result Comparison And Conclusions:

In this work, we predict which bus trips had congestion and which didn't.

As there is uncertainty in some of the records, we handled these records in different ways.

We applied the first method (complete case analysis) directly on the raw data. This is the simplest way to deal with missing\uncertain data, however we lose some of the records in this process.
The second and third methods (max likelihood and VDA using Conditional Mean) where applied on the aggregated data. In both methods we keep all the records, and try to deal with the uncertainty.
The differences between them is that the maximum likelyhood is a model based method were as VDA completes the missing data directly.

We chose the methods after reading the 'Missing Data Problem in Predictive Analytics' article, which provides a literature review of the different methods and when each method is suiteable.
We conclude that VDA and maximum likelyhood require relatively low computing resources and are suitable for numerical features - both important aspects in our case. 

The paper also discusses using 'hybrid' methods which are more suitable for specific tasks, when there is additional knowledge about the features.
As we know that different journey patterns can have different trip times, we replace the traditional VDA (which uses the unconditional mean) with VDA that uses the conditional mean, which is more appropriate in our case.  

We perform a comparison between part 1, in which we do not deal with the uncertain records in any way meaning we use all the data, with the three methods we described above. 

The raw data includes ~237 million records in total, and after aggregation we got 1,414,012 different bus trips. (for part 1, method 2 and method 3).

For the complete case analysis we drop from the raw data ~630K records, and after aggregation we left with 1,411,472 different bus trips. (for method 1).

To check the model performance (by accuracy and recall) on unseen data we split the records into train and test (70%-30% accordingly).

We train the model on the train set, and compute the metrics on both train and test sets. The whole process took a couple of hours, for each method.

In [61]:
results_info = pd.DataFrame([['part1 (all data)', 0.7970904191752517,0.7970539272562175,0.06747116964374761, 0.06784939249414564], 
                             ['method1 (complete only)',0.8041417184534502,0.8040435498036523,0.16204197700883588, 0.16297656202190808],
                             ['method2 (Max LL)',0.796903658458911,0.7971649094486203, 0.06903984015690634, 0.06875790973988474], 
                             ['method3 (DVA)',0.7977002080897876,0.7968390073939433, 0.07407952484823001, 0.07283149390728315]], 
                            columns=['Method','Train Accuracy','Test Accuracy', 'Train Recall','Test Recall']).round(5)
results_df = spark.createDataFrame(results_info)
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
part1 (all data),0.79709,0.79705,0.06747,0.06785
method1 (complete only),0.80414,0.80404,0.16204,0.16298
method2 (Max LL),0.7969,0.79716,0.06904,0.06876
method3 (DVA),0.7977,0.79684,0.07408,0.07283


In [62]:
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
part1 (all data),0.79709,0.79705,0.06747,0.06785
method1 (complete only),0.80414,0.80404,0.16204,0.16298
method2 (Max LL),0.7969,0.79716,0.06904,0.06876
method3 (DVA),0.7977,0.79684,0.07408,0.07283


In [63]:
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
part1 (all data),0.7970904191752517,0.7970539272562175,0.06747116964374761,0.06784939249414564
method1 (complete only),0.8041417184534502,0.8040435498036523,0.16204197700883588,0.16297656202190808
method2 (Max LL),0.796903658458911,0.7971649094486203,0.06903984015690634,0.06875790973988474
method3 (DVA),0.7977002080897876,0.7968390073939433,0.07407952484823001,0.07283149390728315


As we see in the table and graphs above, all methods provide roughly the same results. (Hover with mouse on bar to see exact values)

Method 1 (complete case) acheives the best recall and accuracy, however take into account that the data on which it was trained and tested on is slightly different, because we drop the uncertain records. 

Additionally as the aggregation is done only on the certain records, if the uncertain raw record was with congestion=true, then this can even lead to a different class label for the bus trip.
Thus, as the feature values and labels can change, the measures can change too even without improving the model, and comparison to part1, method2 and method3 should be done with care.

Comparing method 3 (VDA) to part1 (all data) and method2 (Max likelyhood) we see that DVA achieves the highest recall, on both train and test sets, but not higher accuracy.

We see that the acheived accuracy, for both train and test sets and for all methods, roughly matches the percent of records that have isCongestion of 0 out of all the records. 

So, our model does not perform much better than a 'dummy' classifier.

In the final part of the project we aim to improve the performance by using a more complex model, hyper parameter tuning or better dealing with the imbalanced data (e.g weigh differently the two classes or  up-sampling).